Name: Başar Demir

Student Number: 150180080

In [ ]:
!pip install ninja
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.10.2+cu113 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from argparse import Namespace
import numpy as np
import torch
import sys
import dlib

sys.path.append(".")
sys.path.append("..")
sys.path.append('/content/drive/MyDrive/PSPNet')

from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms

#test_opts = TestOptions().parse()

test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None, mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False, test_batch_size=2, test_workers=2)


test_opts.checkpoint_path = '/content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt'


# update test options with options used during training
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)

net = pSp(opts)
net.eval()
net.cuda()
#Prepare the PSPNet + StyleGAN network

transform = transforms.Compose([
	transforms.Resize(size=(256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
#Preprocessing operations for each image

predictor = dlib.shape_predictor("/content/drive/MyDrive/PSPNet/shape_predictor_68_face_landmarks.dat")
#Predict facial landmarks

aligned_first_image = align_face(filepath="/content/drive/MyDrive/PSPNet/nuri.jpg", predictor=predictor)
aligned_first_image = aligned_first_image.convert("RGB")

aligned_second_image = align_face(filepath="/content/drive/MyDrive/PSPNet/face.jpg", predictor=predictor)
aligned_second_image = aligned_second_image.convert("RGB")


from_im = transform(aligned_first_image).unsqueeze(0)
to_im = transform(aligned_second_image).unsqueeze(0)
# Do preprocessing on the aligned face

frames = []
frame_count = 200

with torch.no_grad():
	input = from_im.float().cuda()
	output = to_im.float().cuda()

	_, from_latent_vector = net(input, randomize_noise=False, resize=False, return_latents=True)
	_, to_latent_vector = net(output, randomize_noise=False, resize=False, return_latents=True)
	# Obtain features from input image

	for i in range(frame_count+1):
		latent_vector = ((from_latent_vector * (frame_count-i)) + (to_latent_vector*i))/frame_count

		result, _ = net.decoder([latent_vector.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
		# Feed features to the StyleGAN

		result = result.squeeze().permute((1,2,0)).cpu().numpy()

		result[result>1] = 1
		result[result<-1] = -1
		result = (255*(result+1)//2).astype(np.uint8)
	
		frames.append(result)

Loading pSp from checkpoint: /content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt


In [ ]:
import moviepy.editor as mpy

clip = mpy.ImageSequenceClip(frames , fps = 25)
clip.write_videofile("/content/drive/MyDrive/PSPNet/part-3.mp4", codec="libx264")

[MoviePy] >>>> Building video /content/drive/MyDrive/PSPNet/part-2.mp4
[MoviePy] Writing video /content/drive/MyDrive/PSPNet/part-2.mp4


100%|██████████| 202/202 [00:11<00:00, 18.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/PSPNet/part-2.mp4 

